# Functions

## Packages

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets

from shapely.geometry import Point
import pandas as pd
#import csv

from tqdm import tqdm

import numpy as np
import json, requests 
import pandas as pd
from pandas.io.json import json_normalize

import matplotlib.pyplot as plt
import seaborn as sns
from math import radians, cos, sin, asin, sqrt

import spacy
#!python -m spacy download en_core_web_lg

spacy_nlp = spacy.load("en_core_web_lg")

import re

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2'  )

import unicodedata
from metaphone import doublemetaphone
from fuzzywuzzy import fuzz
from difflib import SequenceMatcher
import re

import geopandas as gpd
from geopandas import GeoDataFrame


2023-03-07 10:29:53.394854: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 10:29:53.636252: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 10:29:53.636300: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-07 10:29:56.267080: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
## load dictionary with the institutions: 

f = open("/home/emma_scharfmann/LeeFleming/MattMarx_Ryan_dataset/institutions2.json","r")
import json
dic_institutions = json.load(f)

## General functions

In [4]:
def URL(base_URL , entity_type , filters):
    url = base_URL + entity_type + filters 
    return url


def get_data(url):
    url = requests.get(url)
    text = url.text
    import json
    data = json.loads(text)
    return data

## encoding the abstract


def reconstruction_abstract(abstract_inverted_index):
    # return the abstract is the abstract exists in the database, else, return None
    
    if abstract_inverted_index != None:
        
        list_values = list(abstract_inverted_index.values())
        list_keys = list(abstract_inverted_index.keys())
        #from the words in the abstract (keys of abstract_inverted_index) and their position in the text (values of abstract_inverted_index), reconstruct the abstract
        
        size_abstract = max([ max(elem) for elem in abstract_inverted_index.values() ] )
        
        abstract = [""]*(size_abstract +1)
        
        for i in range(len(list_values)):
            for pos in list_values[i]:
                abstract[pos] = list_keys[i]
        
        return " ".join(list(abstract))
    
    else:
        return None
            
    
## calculate efficiently the dot product between two vectors

def norm(vector):
    return sqrt(sum(x * x for x in vector))    

def cosine_similarity2(vec_a, vec_b):
        norm_a = norm(vec_a)
        norm_b = norm(vec_b)
        dot = sum(a * b for a, b in zip(vec_a, vec_b))
        return dot / (norm_a * norm_b)

### Extracted texts

In [5]:
def print_extracted_text(name_file):

    file = open("/home/emma_scharfmann/LeeFleming/Synapse_project/" + name_file + ".txt", "r")
    lines = file.readlines()
    count = 0
    for index, line in enumerate(lines):
        read_line = line.strip()
        print(read_line)

    file.close()
    
    
    
def details(name_file , display):
    
    file = open("/home/emma_scharfmann/LeeFleming/Synapse_project/" + name_file + ".txt", "r")
    lines = file.readlines()

    mark = 0 
    dic_details = {}
    count = -1
    for index, line in enumerate(lines):

        line = line.strip()
        if line == "Close explanation":
            break

        if line != "" and  (line[0].isnumeric() and ">" in line and " 	" in line) :
            count += 1


        if mark == 1 and line != "" and line[0] == "*":
            
            if display == True:
                print(count)
                print(text)
                print(" ")
            dic_details[count] = text
            mark = 0


        if mark == 1:
            text = text + line + " "

        if line.split(" 	")[-1] == "Details":
            mark = 1
            text = ""
            
    return dic_details




def key_initiatives(name_file , display ):
    
    file = open("/home/emma_scharfmann/LeeFleming/Synapse_project/" + name_file + ".txt", "r")
    lines = file.readlines()

      
    mark = 0 
    dic_key_initiatives = {}
    count = -1
    for index, line in enumerate(lines):

        line = line.strip()
        if line == "Close explanation":
            break

        if line != "" and  (line[0].isnumeric() and ">" in line and " 	" in line) :

            count += 1



        if mark == 1 and line != "" and ( (line[0].isnumeric() and  ">" in line and " 	" in line) or line == "*Deployment targets:*" or line == "*Announced development targets:*"):
            if display == True: 
                print(count)
                print(text)
                print(" ")
            
            dic_key_initiatives[count] = text
            mark = 0


        if mark == 1:
            text = text + line + " "

        if line == "*Key initiatives:*":


            mark = 1
            text = ""
            
    return dic_key_initiatives




def deployment_target(name_file , display):
    
    file = open("/home/emma_scharfmann/LeeFleming/Synapse_project/" + name_file + ".txt", "r")
    lines = file.readlines()


    mark = 0 
    dic_target = {}
    count = -1
    for index, line in enumerate(lines):

        line = line.strip()
        if line == "Close explanation":
            break

        if line != "" and  (line[0].isnumeric() and ">" in line and " 	" in line) :
            count += 1



        if mark == 1 and line != "" and  ((line[0].isnumeric() and ">" in line and " 	" in line)  or line == "*Announced cost reduction targets:*" or line == "*Announced development targets:*"):
            
            if display == True:
                print(count)
                print(text)
                print(" ")
                
            dic_target[count] = text
            mark = 0


        if mark == 1:
            text = text + line + " "

        if line == "*Deployment targets:*" or line == "*Announced development targets:*":

            mark = 1
            text = ""
            
    return dic_target


 
    
def cost_reduction_target(name_file , display):
    
    file = open("/home/emma_scharfmann/LeeFleming/Synapse_project/" + name_file + ".txt", "r")
    lines = file.readlines()
    
    mark = 0 
    dic_cost = {}
    count = -1
    for index, line in enumerate(lines):

        line = line.strip()
        
        if line == "Close explanation":
            break

        if line != "" and  (line[0].isnumeric() and ">" in line and " 	" in line) :


            count += 1



        if mark == 1 and line != ""  and (line[0].isnumeric() and ">" in line and " 	" in line) :
            
            if display == True: 
                print(count)
                print(text)
                print(" ")
                
            dic_cost[count] = text
            mark = 0


        if mark == 1:
            text = text + line + " "

        if line == "*Announced cost reduction targets:*":

            mark = 1
            text = ""
            
    return dic_cost





def key_words(name_file, display ):
    
    file = open("/home/emma_scharfmann/LeeFleming/Synapse_project/" + name_file + ".txt", "r")
    lines = file.readlines()
    
    list_categories = []
    count = -1
    for index, line in enumerate(lines):

        line = line.strip()
        
        if line == "Close explanation":
            break

        if line != "" and  (line[0].isnumeric() and ">" in line and " 	" in line) :
            count += 1
            
            if display == True:
                print("Technologies"  , count+1 , ":")

        if line != "":

            if line[0].isnumeric() and ">" in line and " 	" in line:
                i = 0
                try:
                    line = line.split(" 	")[2]
                except:
                    print(line)
                    break
                
                if "Details" not in lines[index] and "Moderate" not in lines[index]:
                
                    while "	" not in line:
                        i += 1
                        if "Details"==lines[index + i][:7] or "End-use"==lines[index + i][:7]:
                            break
                        else:
                            line = line + "  " +  lines[index + i]

                #if " Production" in line:
                    #line = line.replace(" Production" , "")

                line = line.replace("\n" , " ")
                line = line.replace("/" , " ")
                line = line.replace("-" , " ")
                line = line.split(" 	")[0]

                if "  " in line:
                    line = line.replace("  ", " ")
                line = line.split(">")


                if "(" in line[-1]:
                    line[-1] = line[-1].split("(")[0] 


                for i in range(len(line)):

                    # remove multiple spaces
                    line[i] = re.sub(' +', ' ', line[i])
                    # remove trailing spaces
                    line[i] = line[i].strip()

    

                if display == True:
                    print(line)
                    print(" ")
                    
                if '' in line:
                    line.remove('')

                list_categories.append([count , line])
                
    return list_categories



def technology(name_file, display ):
    
    file = open("/home/emma_scharfmann/LeeFleming/Synapse_project/" + name_file + ".txt", "r")
    lines = file.readlines()
    
    list_categories = []
    count = -1
    for index, line in enumerate(lines):

        line = line.strip()
        
        if line == "Close explanation":
            break

        if line != "" and  (line[0].isnumeric() and ">" in line and " 	" in line) :
            count += 1
            
            if display == True:
                print("Technologies"  , count+1 , ":")

        if line != "":

            if line[0].isnumeric() and ">" in line and " 	" in line:
                i = 0
                try:
                    line = line.split(" 	")[1]
                except:
                    print(line)
                    break
                

                line = line.replace("\n" , " ")
                line = line.replace("/" , " ")
                line = line.replace("-" , " ")
                line = line.strip()
                line = re.sub(' +', ' ', line)
                line = line.split(" 	")[0]
                line = line.split(">")


                if "(" in line[-1]:
                    line[-1] = line[-1].split("(")[0] 


                for i in range(len(line)):

                    # remove multiple spaces
                    line[i] = re.sub(' +', ' ', line[i])
                    # remove trailing spaces
                    line[i] = line[i].strip()

    

                if display == True:
                    print(line)
                    print(" ")
                

                list_categories.append([count , line])
                
    return list_categories


In [6]:
dic_details = details("iea" , False)
dic_key_words = key_words("iea" , False)
list_category = technology("iea" , False)

In [7]:
list_category[15]

[15, ['Transport', 'Shipping']]

In [8]:
dic_details[15]

' Automated and connected ships may improve energy efficiency. Fully autonomous ships may be able to take longer voyages at lower speeds and greater fuel efficiency.  '

## Functions papers 

### Get related papers

In [9]:
def related_papers(number_technology , list_categories , carbon_related , display  , key_words):

    dic = {}

    max_count = 0
    base_URL_OA = f'https://api.openalex.org/'
    filter_works = f'works?'
    for element in list_categories[number_technology:number_technology+1]:
        

        filter_openalex = f"search=" + key_words


        if carbon_related == True:
            filter_openalex += "&filter=concepts.id:https://openalex.org/C132651083|https://openalex.org/C115343472|https://openalex.org/C530467964&per_page=200&mailto=emma_scharfmann@berkeley.edu"
        
        else:
            filter_openalex += "&per_page=200&mailto=emma_scharfmann@berkeley.edu"
            
            
        filter_openalex = filter_openalex.replace(" " , "%20")
        
        url = URL(base_URL_OA , filter_works, filter_openalex) 
        data = get_data(url)
        count = data["meta"]["count"] 



        if display == True:
            print(  data["meta"]["count"] , [elem[i] for i in range(1,len(elem))]  )
            print(url)
        
        for i in range(len(data["results"])):
            dic[ data["results"][i]["id"]] = {}
            dic[ data["results"][i]["id"]]["title"] = data["results"][i]["title"]
            dic[ data["results"][i]["id"]]["abstract"] = reconstruction_abstract(data["results"][i]["abstract_inverted_index"])
            dic[ data["results"][i]["id"]]["concepts"] = data["results"][i]["concepts"]
            dic[ data["results"][i]["id"]]["date"] = data["results"][i]["publication_date"]
            dic[ data["results"][i]["id"]]["authorships"] = data["results"][i]["authorships"]
            dic[ data["results"][i]["id"]]["cited_by_count"] = data["results"][i]["cited_by_count"]

            if len(data["results"][i]["authorships"]) > 0:
                if data["results"][i]["authorships"][0]["institutions"] != []:
                    dic[ data["results"][i]["id"]]["countries"] = data["results"][i]["authorships"][0]["institutions"][0]["country_code"]
                    dic[ data["results"][i]["id"]]["institutions"] = data["results"][i]["authorships"][0]["institutions"][0]["display_name"]
                else:
                    dic[ data["results"][i]["id"]]["countries"] = ""
                    dic[ data["results"][i]["id"]]["institutions"] = ""
                dic[ data["results"][i]["id"]]["authors"] = data["results"][i]["authorships"][0]["author"]["display_name"]

                for j in range(1 , len(data["results"][i]["authorships"])):
                    if data["results"][i]["authorships"][j]["institutions"] != []:
                        dic[ data["results"][i]["id"]]["institutions"] += ", " + data["results"][i]["authorships"][j]["institutions"][0]["display_name"]
                        dic[ data["results"][i]["id"]]["countries"] = data["results"][i]["authorships"][j]["institutions"][0]["country_code"]
                    dic[ data["results"][i]["id"]]["authors"] += ", " + data["results"][i]["authorships"][j]["author"]["display_name"]

    if display == True: 
        print(" ")
        
    return dic
        
    



### Rank the papers

In [10]:
def ranking_papers(number_technology,  dic_details , dic ,  list_categories , display , size ):
    dic_scores_papers = {}
    

    if display == True:
        print("Technology " + str(number_technology))
        print("Key words: " , list_categories[number_technology][1])
            
    if number_technology in dic_details:
        reference_text = dic_details[number_technology]
            
        if display== True:
            print("Technology details: " , reference_text)
            print(" ")
        encoded_text = model.encode(reference_text, convert_to_tensor=False).tolist()
        
        
        if len(dic) == 0:
            return "No paper found, select other key words" 
        
        
        for ids in list(dic.keys()):

            dic_scores_papers[ids] = {}
            
            if dic[ids]["title"] != None:
                encoded_title = model.encode(dic[ids]["title"], convert_to_tensor=False).tolist()
                score_title = cosine_similarity2(encoded_title, encoded_text)
            else:
                score_title = None

        

            if dic[ids]["abstract"] != None:
                encoded_abstract = model.encode(dic[ids]["abstract"], convert_to_tensor=False).tolist()
                score_abstract = cosine_similarity2(encoded_abstract, encoded_text)
            else:
                score_abstract = None


            #concepts = ""
            #for elem in dic[ids]["concepts"]:
            #    concepts += elem["display_name"] + " "
            #encoded_concepts = model.encode(concepts, convert_to_tensor=False).tolist()
            #score_concepts = cosine_similarity2(encoded_concepts, encoded_text)

            dic_scores_papers[ids]["title comparision"] = score_title
            dic_scores_papers[ids]["abstract comparision"] = score_abstract
            #dic_scores_papers[ids]["concepts comparision"] = score_concepts
            
            dic_scores_papers[ids]["title"] = dic[ids]["title"]
            dic_scores_papers[ids]["citations"] = dic[ids]["cited_by_count"]
            

            dic_scores_papers[ids]["date"] = dic[ids]["date"][:4]
            if "institutions" in dic[ids]:
                dic_scores_papers[ids]["institutions"] = dic[ids]["institutions"]
            #    dic_scores_papers[ids]["countries"] = dic[ids]["countries"]
                
            else:
                dic_scores_papers[ids]["institutions"] = None
            #    dic_scores_papers[ids]["countries"] = None
            
            if "authors" in dic[ids]:
                dic_scores_papers[ids]["number of co-authors"] = len(dic[ids]["authors"].split(","))
                dic_scores_papers[ids]["authors"] = dic[ids]["authors"]
            
            

        return dic_scores_papers
        


    else:
        return "No description given"





In [11]:
def get_ranking_related_papers( technologies , number_technology , carbon_related , size , research_words):
    name_file = "iea"
    list_categories = key_words( name_file, False)
    dic_details = details(name_file , False)
    dic_related_papers = related_papers(number_technology , list_categories , carbon_related , False , research_words)
    dic_scores_papers = ranking_papers(number_technology,  dic_details , dic_related_papers ,  list_categories , False , size  )
    
    if dic_scores_papers == {}:
        return "No paper found"
    
    elif type(dic_scores_papers) == str:
        return dic_scores_papers
    
    else:
        return pd.DataFrame(dic_scores_papers).T.sort_values(by="abstract comparision" , ascending = False)[:size]
     
    


### Extract quantitative data

In [12]:
def extract_sentences_with_numbers(text , text_name):
    if text != None:
        text = text.replace("CO 2" , "CO2")
        text = text.replace("CO 3" , "CO3")
        text = text.replace("CO(2)" , "CO2")
        text = text.replace("CO(3)" , "CO3")
        
        print(text_name + ": " , text)
        print(" ")



        list_text = list(text)
        for i in range(1,len(list_text)-1):
            if list_text[i] == " " and list_text[i-1] == "." and list_text[i+1].isupper():
                list_text[i] = "~"

        text = "".join(list_text)
        text = text.split("~")
        for sentence in text:                

            if any(char.isdigit() for char in sentence):
                
                
                
                if "CO2" in sentence:
                    print("\x1b[31mCARBON RELATED:\x1b[0m", sentence)
                    print(" ")


                    
                if "GJ" in sentence or "MJ" in sentence:
                    print("\x1b[31mENERGY:\x1b[0m" , sentence)
                    print(" ")


                    
                ##price
                if "€"  in sentence or "$" in sentence or "EUR" in sentence or "dollars" in sentence.lower():
                    print("\x1b[31mPRICE:\x1b[0m" , sentence)
                    print(" ")


                    
                ##dates
                digits = []
                for word in sentence.replace("," , "").replace("%" , "").replace("." , " ").split():
                    if word.isdigit() and 1850 < int(word) < 2200 :
                        digits.append(word)
                if digits != []:
                    print("\x1b[31mDATE:\x1b[0m" , sentence)
                    print(" ")


                    
                ##CO quantity
                if "Mt" in sentence or "tC" in sentence or "t-C" in sentence:
                    print("\x1b[31mCARBON QUANTITY:\x1b[0m" , sentence)
                    print(" ")
 
    print(" ")
        
 
 
    
    
def extract_quantitative_data_technology(technologies, number_technology):
    count = 0
      
    name_file = "iea"
    dic_target = deployment_target(name_file , False)
    dic_cost = cost_reduction_target(name_file , False)
    dic_details = details(name_file , False)
    
    
    if number_technology in dic_details:
        reference_text = dic_details[number_technology]
            
        print("\033[96mFROM IEA website: ")
        print("\033[92mTechnology details: \x1b[0m" , reference_text)
        print(" ")
        encoded_text = model.encode(reference_text, convert_to_tensor=False).tolist()

    if number_technology in dic_target:
        cost_target_text = dic_target[number_technology]
        print("\033[96mFROM IEA website: ")
        sentences = extract_sentences_with_numbers(cost_target_text , "\033[92mDeployment target and Announced development target\x1b[0m")

    if number_technology in dic_cost:
        cost_text = dic_cost[number_technology]
        print("\033[96mFROM IEA website: ")
        sentences = extract_sentences_with_numbers(cost_text , "\033[92mAnnounced cost reduction targets\x1b[0m")


        
        
        
def extract_quantitative_data_paper( work_id):
    

    try:
        url = "https://api.openalex.org/works/" + str(work_id)
        data = get_data(url)
        date = data["publication_date"]
        title = data["title"]
        abstract = reconstruction_abstract(data["abstract_inverted_index"])
        concepts = ", ".join( [elem["display_name"] for elem in data["concepts"]] )
        authors = ", ".join( [elem["author"]["display_name"] for elem in data["authorships"]] )
        institutions = ", ".join( set([elem["institutions"][0]["display_name"] for elem in data["authorships"]]) )
        print("\033[96mFROM OpenAlex: ")
        print("\033[92mPaper link: "  + url)
        print(" ")
        print("\033[92mTitle: \x1b[0m"  + title)
        print(" ")
        print("\033[92mConcepts: \x1b[0m"  + concepts)
        print(" ")
        print("\033[92mDate: \x1b[0m"  + date)
        print(" ")
        print("\033[92mAuthors: \x1b[0m"  + authors)
        print(" ")
        print("\033[92mInstitutions: \x1b[0m"  + institutions)
        print(" ")
        sentences = extract_sentences_with_numbers(abstract , "\033[92mAbstract\x1b[0m")
    except:
        print("Enter a valid work id from OpenAlex")
    





### Get papers related to cited projects

In [13]:
def related_projects(technologies, number_technology , carbon_related , size):

    name_file = "iea"
    dic_details = details(name_file , False)
    dic_key_initiatives = key_initiatives(name_file , False )
    
    
    dic2 = {}

    reference_text = dic_details[number_technology]
    print("\033[92mTechnology details: \x1b[0m" , reference_text)
    print(" ")
    encoded_text = model.encode(reference_text, convert_to_tensor=False).tolist()

    if number_technology in dic_key_initiatives:
        key_initiative = dic_key_initiatives[number_technology]
        print("\033[92mTechnology key initiatives: \x1b[0m" ,  key_initiative)

        location_entities = set()
        doc = spacy_nlp(key_initiative.strip())
        for j in doc.ents:
            entry = str(j.lemma_).lower()
            key_initiative = key_initiative.replace(str(j).lower(), "")
            if j.label_ in ["ORG"]:
                location_entities.add(j)
        print(" ")
        print("\033[92mExtracted projects and organizations names: \x1b[0m", location_entities)


        base_URL_OA = f'https://api.openalex.org/'
        filter_works = f'works?'
        for element in location_entities:

            filter_openalex = f"search=" 
            filter_openalex += str(element) 
            if carbon_related == True:
                filter_openalex += "&filter=concepts.id:https://openalex.org/C2780021121&per_page=200&mailto=emma_scharfmann@berkeley.edu"
            else:
                filter_openalex += "&per_page=200&mailto=emma_scharfmann@berkeley.edu"

            filter_openalex = filter_openalex.replace(" " , "%20")

            url = URL(base_URL_OA , filter_works, filter_openalex) 

            data = get_data(url)
            count = len(data)
            if count < 50:

                for k in range(count):
                    
                    if data["results"][k]["abstract_inverted_index"] != None:
                        encoded_abstract = model.encode(reconstruction_abstract(data["results"][k]["abstract_inverted_index"]) ,  convert_to_tensor=False).tolist()
                        abstract_similarity = cosine_similarity2(encoded_abstract, encoded_text)
                    else:
                        abstract_similarity = None
                        
                    if data["results"][k]["title"] != None:
                        encoded_title = model.encode(data["results"][k]["title"] ,  convert_to_tensor=False).tolist()
                        title_similarity = cosine_similarity2(encoded_title, encoded_text)
                    else:
                        title_similarity = None
                        
                    work_id = data["results"][k]["id"]
                    dic2[work_id] = {}
                    dic2[work_id]["project/institution name"] = str(element)
                    dic2[work_id]["title comparision"] = title_similarity
                    dic2[work_id]["abstract comparision"] = abstract_similarity

            else:

                key_word = list_categories[7]
                elem = key_word[1]

                string = ""
                for j in range(1,len(elem)):
                    if elem[j] == "CCUS":
                        elem[j] = "Carbon capture and storage"
                    string += elem[j]  + " " 


                string += str(element)
                filter_openalex = f"search="  + string 

                if carbon_related == True:
                    filter_openalex += "&filter=concepts.id:https://openalex.org/C2780021121&per_page=200&mailto=emma_scharfmann@berkeley.edu"
                else:
                    filter_openalex += "&per_page=200&mailto=emma_scharfmann@berkeley.edu"
                filter_openalex = filter_openalex.replace(" " , "%20")
                url = URL(base_URL_OA , filter_works, filter_openalex) 
                data = get_data(url)
                count = data["meta"]["count"]
                for k in range(count):
                    
                    if data["results"][k]["abstract_inverted_index"] != None:
                        encoded_abstract = model.encode(reconstruction_abstract(data["results"][k]["abstract_inverted_index"]) ,  convert_to_tensor=False).tolist()
                        abstract_similarity = cosine_similarity2(encoded_abstract, encoded_text)
                    else:
                        abstract_similarity = None
                        
                    if data["results"][k]["title"] != None:
                        encoded_title = model.encode(data["results"][k]["title"] ,  convert_to_tensor=False).tolist()
                        title_similarity = cosine_similarity2(encoded_title, encoded_text)
                    else:
                        title_similarity = None
                        
                    work_id = data["results"][k]["id"]
                    dic2[work_id] = {}
                    dic2[work_id]["project/institution name"] = str(element)
                    dic2[work_id]["title comparision"] = title_similarity
                    dic2[work_id]["abstract comparision"] = abstract_similarity
                    


        if dic2 != {}:
            return pd.DataFrame(dic2).T.sort_values(by="abstract comparision" , ascending = False)[:size]
        else:
            return "No paper found"

    else:
        return "No key initiatives given"

### Main authors

In [14]:
#merge the nobiliary particles with the last name
#ln_suff file can be modified if more or less nobiliary particles want to be suppressed

with open("/home/emma_scharfmann/LeeFleming/MattMarx_Ryan_dataset/Patent_paper_pairs/ln_suff.json","r", encoding="utf-8") as f:
    ln_suff = json.load(f)
    
def ln_suff_merge(string):
    for suff in ln_suff:
        if f" {suff} " in string or string.startswith(f"{suff} "):
            return string.replace(f"{suff} ",suff.replace(" ",""))
    return string



#suppress all the unwanted suffixes from a string. 
#name_del file can be modified if more or less suffixes want to be suppressed 

with open("/home/emma_scharfmann/LeeFleming/MattMarx_Ryan_dataset/Patent_paper_pairs/name_del-2.json","r", encoding="utf-8") as f:
    name_del = json.load(f)

def name_delete(string):
    for elmt in name_del:
        if f" {elmt}" in string:
            return string.replace(f" {elmt}","")
    return string



#normalize a string dat that represents often a name. 

def normalize(data):
    normal = unicodedata.normalize('NFKD', data).encode('ASCII', 'ignore')
    val = normal.decode("utf-8")
    # delete unwanted elmt
    val = name_delete(val)
    # lower full name in upper
    val = re.sub(r"[A-Z]{3,}", lambda x: x.group().lower(), val)
    # add space in front of upper case
    val = re.sub(r"(\w)([A-Z])", r"\1 \2", val)
    # Lower case
    val = val.lower()
    # remove special characters
    val = re.sub('[^A-Za-z0-9 ]+', ' ', val)
    # remove multiple spaces
    val = re.sub(' +', ' ', val)
    # remove trailing spaces
    val = val.strip()
    # suffix merge
    val = ln_suff_merge(val)

    return val


def main_authors(technologies,number_technology , carbon_related , size , research_words):
    
    name_file = "iea"
    list_categories = key_words( name_file, False)
    dic_details = details(name_file , False)
    dic_papers = related_papers(number_technology , list_categories , carbon_related , False , research_words)
    
    dic_papers_co_authors = {}
        


    for paper in dic_papers:

        for k in range(len(dic_papers[paper]["authorships"])):
            coauthor_id = dic_papers[paper]["authorships"][k]["author"]["id"]
            
            
            author_name = dic_papers[paper]["authorships"][k]["author"]["display_name"]
            author_name_norm = normalize(dic_papers[paper]["authorships"][k]["author"]["display_name"]).split()
            if len (author_name_norm) > 0:
                author_name_norm = author_name_norm[0] + " " + author_name_norm[-1]

                if author_name_norm not in dic_papers_co_authors:
                    dic_papers_co_authors[author_name_norm] = {}
                    dic_papers_co_authors[author_name_norm]["Author's name(s)"] = author_name
                    dic_papers_co_authors[author_name_norm]["Author's id(s)"] = coauthor_id[21:]
                    dic_papers_co_authors[author_name_norm]["Number of occurence within the 100 most related papers"] = 1
                    dic_papers_co_authors[author_name_norm]["Number of related citations"] = dic_papers[paper]["cited_by_count"]
                else:
                    dic_papers_co_authors[author_name_norm]["Number of occurence within the 100 most related papers"] += 1
                    if author_name not in dic_papers_co_authors[author_name_norm]["Author's name(s)"]:
                        dic_papers_co_authors[author_name_norm]["Author's name(s)"] += ", " + author_name[0]
                    if coauthor_id[21:] not in dic_papers_co_authors[author_name_norm]["Author's id(s)"]:                                                              
                        dic_papers_co_authors[author_name_norm]["Author's id(s)"] += ", " + coauthor_id[21:]
                        dic_papers_co_authors[author_name_norm]["Number of related citations"] += dic_papers[paper]["cited_by_count"]



    if dic_papers_co_authors != {}:
        dic_papers_co_authors = {k: v for k, v in sorted(dic_papers_co_authors.items(), key=lambda item: item[1]["Number of occurence within the 100 most related papers"] , reverse = True)}

        for author_name_norm in list(dic_papers_co_authors.keys())[:size]:
            list_ids = dic_papers_co_authors[author_name_norm]["Author's id(s)"].split(", ")
            work_count = 0
            cited_by_count = 0
            institutions = ''
            for elem in list_ids:
                if len(elem) > 3:
                    
                    data = get_data("https://api.openalex.org/people/" + elem)
                    work_count += data["works_count"]
                    cited_by_count += data["cited_by_count"]
                    institutions += data["last_known_institution"]["display_name"]
                    
            dic_papers_co_authors[author_name_norm]["Last Known Institution"] = institutions
            dic_papers_co_authors[author_name_norm]["Number of works"] = work_count
            dic_papers_co_authors[author_name_norm]["Number of citations"] = cited_by_count

            
        return pd.DataFrame(dic_papers_co_authors, index = [ "Author's name(s)" , "Author's id(s)" ,  "Number of occurence within the 100 most related papers" , "Last Known Institution" , "Number of works" , "Number of citations" ,  "Number of related citations"]).T[:size].style.hide(axis="index")

    else:
        return ("Select another category")

In [15]:
def map_authors(technologies, number_technology , carbon_related , size , category):

    name_file = "iea"
    list_categories = key_words( name_file, False)
    dic_details = details(name_file , False)
    dic_papers = related_papers(number_technology , list_categories , carbon_related , False , category)

    reference_text = dic_details[number_technology]
    
    encoded_text = model.encode(reference_text, convert_to_tensor=False).tolist()
    
    dic_papers_co_authors = {}
    count = 0
    

    for paper in list(dic_papers.keys())[:30]:
        
        if dic_papers[paper]["abstract"] != None:
            encoded_abstract = model.encode(dic_papers[paper]["abstract"], convert_to_tensor=False).tolist()
            score_abstract = cosine_similarity2(encoded_abstract, encoded_text)
        else:
            score_abstract = None

        for k in range(len(dic_papers[paper]["authorships"])):
            
            if dic_papers[paper]["authorships"][k]["institutions"] != [] and "id" in dic_papers[paper]["authorships"][k]["institutions"][0]:
                institution_id = dic_papers[paper]["authorships"][k]["institutions"][0]["id"]
                if institution_id in dic_institutions:
                    data = dic_institutions[institution_id]

                    dic_papers_co_authors[count] = {}
                    dic_papers_co_authors[count]["Longitude"] = data["geo"]["longitude"]
                    dic_papers_co_authors[count]["Latitude"] = data["geo"]["latitude"]
                    dic_papers_co_authors[count]["abstract comparision"] = score_abstract
                    count += 1
                        
    if dic_papers_co_authors == {}:
        return "No papers"
    

    
    map_df = pd.DataFrame(dic_papers_co_authors,).T.sort_values(by="abstract comparision" , ascending = False)[:size]
    map_df['Longitude']=map_df['Longitude'].astype(float)
    map_df['Latitude']=map_df['Latitude'].astype(float)
    

    geometry = [Point(xy) for xy in zip(map_df['Longitude'], map_df['Latitude'])]
    gdf = GeoDataFrame(map_df, geometry=geometry)   

    #this is a simple map that goes with geopandas
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    gdf.plot(ax=world.plot( color='white', edgecolor='black' ), marker='o', color='red', markersize=15 , zorder = 1);
    
    plt.xlim([-180, 180])
    plt.ylim([-90, 90])


    plt.title("Main authors: geographic location" )
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.show()
       

## Functions patents

### Get related patents

In [16]:
def related_patents(number_technology , research_words, carbon_related , display):
    
    name_file = "iea"
    list_categories = key_words( name_file, False)

    dic_patents = {}

    max_count = 0
    base_URL_PV = "https://api.patentsview.org/"
    filter_works = "patents/query?"
    filter_PV = "q={%22_and%22:[{%22_text_all%22:{%22patent_abstract%22:%22" 
    filter_PV += research_words
    
    if carbon_related == True:
        filter_PV += "%22}},{%22_eq%22:{%22cpc_group_id%22:%22Y02E%22}}]}&f=[%22patent_number%22,%22patent_title%22,%22assignee_country%22,%22patent_date%22,%22inventor_id%22,%22assignee_organization%22,%22inventor_longitude%22,%22inventor_latitude%22,%22inventor_last_name%22,%22inventor_first_name%22,%22cpc_subsection_title%22,%22assignee_city%22,%22patent_abstract%22,%22patent_kind%22,%22cpc_group_id%22,%22assignee_organization%22,%22citedby_patent_number%22]"
    
    else:
        filter_PV += "%22}}]}&f=[%22patent_number%22,%22patent_title%22,%22assignee_country%22,%22patent_date%22,%22assignee_organization%22,%22inventor_longitude%22,%22inventor_latitude%22,%22inventor_last_name%22,%22inventor_id%22,%22inventor_first_name%22,%22cpc_subsection_title%22,%22assignee_city%22,%22patent_abstract%22,%22patent_kind%22,%22cpc_group_id%22,%22assignee_organization%22,%22citedby_patent_number%22]"

        
    filter_PV = filter_PV.replace(" " , "%20")

        

    url = URL(base_URL_PV , filter_works, filter_PV) 
    data = get_data(url)


    if display == True:
        print(  data["total_patent_count"] , elem[-1] )
        print(url)



    
    for i in range(data["count"]):
        dic_patents[ "US-" + data["patents"][i]["patent_number"]] = {}
        dic_patents[ "US-" + data["patents"][i]["patent_number"]]["title"] = data["patents"][i]["patent_title"]
        dic_patents["US-" + data["patents"][i]["patent_number"]]["abstract"] = data["patents"][i]["patent_abstract"]

        dic_patents[ "US-" + data["patents"][i]["patent_number"]]["assignee"] = str(data["patents"][i]["assignees"][0]["assignee_organization"])
        dic_patents["US-" + data["patents"][i]["patent_number"]]["assignee_city"] = str(data["patents"][i]["assignees"][0]["assignee_city"])
        dic_patents["US-" + data["patents"][i]["patent_number"]]["assignee_country"] = str(data["patents"][i]["assignees"][0]["assignee_country"])
        for j in range(1, len(data["patents"][i]["assignees"])):
            dic_patents[ "US-" + data["patents"][i]["patent_number"]]["assignee"] +=  ", " + str(data["patents"][i]["assignees"][j]["assignee_organization"])
            dic_patents[ "US-" + data["patents"][i]["patent_number"]]["assignee_city"] +=  ", " + str(data["patents"][i]["assignees"][j]["assignee_city"])
            dic_patents["US-" + data["patents"][i]["patent_number"]]["assignee_country"] +=  ", " + str(data["patents"][i]["assignees"][j]["assignee_country"])

        dic_patents[ "US-" + data["patents"][i]["patent_number"]]["list_inventors"] = data["patents"][i]["inventors"]

        dic_patents[ "US-" + data["patents"][i]["patent_number"]]["inventors"] = str(data["patents"][i]["inventors"][0]["inventor_first_name"]) + " " + str(data["patents"][i]["inventors"][0]["inventor_last_name"])
        for j in range(1, len(data["patents"][i]["inventors"])):
            dic_patents[ "US-" + data["patents"][i]["patent_number"]]["inventors"] +=  ", " + str(data["patents"][i]["inventors"][j]["inventor_first_name"]) + " " + str(data["patents"][i]["inventors"][j]["inventor_last_name"])


        dic_patents["US-" + data["patents"][i]["patent_number"]]["date"] = data["patents"][i]["patent_date"]
        dic_patents["US-" + data["patents"][i]["patent_number"]]["number_citations"] = len(data["patents"][i]["citedby_patents"])


    if display == True:
        print(" ")
            
            
    return dic_patents


### Extract quantitative data

In [17]:
   
def extract_quantitative_data_patent( patent_id):

    try: 
        url = "https://api.patentsview.org/patents/query?q={%22patent_id%22:%22" + str(patent_id) + "%22}&f=[%22patent_number%22,%22patent_title%22,%22patent_abstract%22,%22patent_date%22,%22inventor_last_name%22,%22inventor_first_name%22,%22assignee_organization%22]"
        data = get_data(url)["patents"][0]
        title = data["patent_title"]
        abstract = data["patent_abstract"]
        print("\033[96mFROM PATENTSVIEW:")
        print("\033[92mPatent link: \x1b[0m"  + url)
        print(" ")
        print("\033[92mTitle: \x1b[0m"  + title)
        print(" ")
        print("\033[92mAbstract: \x1b[0m"  + abstract)
        print(" ")
        print("\033[92mDate: \x1b[0m"  + data["patent_date"])
        print(" ")
        print("\033[92mInventors: \x1b[0m"   ,  ", ".join([ data["inventors"][i]["inventor_first_name"] + " " + data["inventors"][i]["inventor_last_name"] for i in range(len(data["inventors"])) ]) )
        print(" ")
        print("\033[92mAssignee: \x1b[0m" , ", ".join( [ str(data["assignees"][i]["assignee_organization"]) for i in range(len(data["assignees"])) ] ) )

    except:
        print("Enter a valid patent_id from PatentsView")
    



### Rank the patents

In [18]:
def ranking_patents(number_technology , research_words, carbon_related , display):
    
    name_file = "iea"
    list_categories = key_words( name_file, False)
    dic_details = details(name_file , False)
    dic_patents = related_patents(number_technology , research_words, carbon_related , display)
    

    dic_scores = {}

    if display == True:
        print("Key words: " , list_categories[number_technology][1])
        
    if number_technology in dic_details:
        reference_text = dic_details[number_technology]
        if display == True:
            print("Technology details: " , reference_text)
            print(" ")
        encoded_text = model.encode(reference_text, convert_to_tensor=False).tolist()
        
        
        if len(dic_patents ) == 0:
            return "Select other key words"
        
        
        else:
            for ids in list(dic_patents.keys()):

                dic_scores[ids] = {}

                encoded_title = model.encode(dic_patents[ids]["title"], convert_to_tensor=False).tolist()
                score_title = cosine_similarity2(encoded_title, encoded_text)


                if dic_patents[ids]["abstract"] != None:
                    encoded_abstract = model.encode(dic_patents[ids]["abstract"], convert_to_tensor=False).tolist()
                    score_abstract = cosine_similarity2(encoded_abstract, encoded_text)
                else:
                    score_abstract = None

                dic_scores[ids]["title comparision"] = score_title
                dic_scores[ids]["abstract comparison"] = score_abstract
                dic_scores[ids]["title"] = dic_patents[ids]["title"]
                dic_scores[ids]["citations"] = dic_patents[ids]["number_citations"]
                dic_scores[ids]["date"] = dic_patents[ids]["date"][:4]
                dic_scores[ids]["assignee"] = dic_patents[ids]["assignee"]
                #dic_scores[ids]["assignee_city"] = dic_patents[ids]["assignee_city"]
                #dic_scores[ids]["assignee_country"] = dic_patents[ids]["assignee_country"]
                dic_scores[ids]["inventors"] = dic_patents[ids]["inventors"]
                dic_scores[ids]["number of co-inventors"] = len(dic_patents[ids]["inventors"].split(","))
    return dic_scores





def get_ranking_patents(technologies, number_technology ,  carbon_related ,category, size):
    dic_patents = ranking_patents(number_technology , category, carbon_related , False)
    if dic_patents == {}:
        return "No patent found, select other key words"

    elif dic_patents == "Select other key words":
        return dic_patents
    
    else:
        return pd.DataFrame(dic_patents).T.sort_values(by="abstract comparison" , ascending = False)[:size]
    

### Main inventors

In [19]:
def main_inventors(technologies, number_technology , carbon_related , category , size ):
    
    dic_patents = related_patents(number_technology , category, carbon_related , display)
    dic_patents_co_inventors = {}
    

    for patent in dic_patents:
        for k in range(len(dic_patents[patent]["list_inventors"])):
            
            inventor_id = dic_patents[patent]["list_inventors"][k]["inventor_id"]
            inventor_name = dic_patents[patent]["list_inventors"][k]["inventor_first_name"] + " " + dic_patents[patent]["list_inventors"][k]["inventor_last_name"]
            inventor_name_norm = normalize(inventor_name).split()
            inventor_name_norm = inventor_name_norm[0] + " " + inventor_name_norm[-1]

            if inventor_name_norm not in dic_patents_co_inventors:
                dic_patents_co_inventors[inventor_name_norm] = {}
                dic_patents_co_inventors[inventor_name_norm]["Inventor's name"] = inventor_name 
                dic_patents_co_inventors[inventor_name_norm]["PatentsView inventor's id"] =  inventor_id 
                dic_patents_co_inventors[inventor_name_norm]["Number of occurence"] = 1
                dic_patents_co_inventors[inventor_name_norm]["Number of related citations"] = dic_patents[patent]["number_citations"]

            else:
                if inventor_id not in dic_patents_co_inventors[inventor_name_norm]["PatentsView inventor's id"] :
                    dic_patents_co_inventors[inventor_name_norm]["PatentsView inventor's id"] += ", " + inventor_id
                if inventor_name not in dic_patents_co_inventors[inventor_name_norm]["Inventor's name"] :
                    dic_patents_co_inventors[inventor_name_norm]["Inventor's name"] += ", " + inventor_name
                dic_patents_co_inventors[inventor_name_norm]["Number of occurence"] += 1
                dic_patents_co_inventors[inventor_name_norm]["Number of related citations"] += dic_patents[patent]["number_citations"]


        dic_patents_co_inventors = {k: v for k, v in sorted(dic_patents_co_inventors.items(), key=lambda item: item[1]["Number of occurence"] , reverse = True)}
    
    if dic_patents_co_inventors == {}:
    
        
        return "No patent, select other key words"
    else:
        
        for inventor_name_norm in list(dic_patents_co_inventors.keys())[:size]:
            list_inventors = dic_patents_co_inventors[inventor_name_norm]["PatentsView inventor's id"].split(", ")
            work_count = 0
            cited_by_count = 0
            
            for elem in list_inventors:
                url = "https://api.patentsview.org/inventors/query?q={%22inventor_id%22:[%22" + elem + "%22]}&f=[%22inventor_total_num_patents%22,%22patent_num_cited_by_us_patents%22]"
                data = get_data(url)["inventors"][0]
                work_count += int(data["inventor_total_num_patents"])
                for k in range(len(data["patents"])):
                    cited_by_count += int(data["patents"][k]["patent_num_cited_by_us_patents"])
                
            dic_patents_co_inventors[inventor_name_norm]["Number of patents"] = work_count
            dic_patents_co_inventors[inventor_name_norm]["Number of US patents citations"] = cited_by_count
            
        
                
                
        return pd.DataFrame(dic_patents_co_inventors , index = ["Inventor's name", "PatentsView inventor's id", "Number of occurence"  , "Number of patents" ,"Number of US patents citations" ,  "Number of related citations"]).T[:size].style.hide(axis="index")

        
        

In [20]:

def map_inventors(technologies, number_technology , carbon_related , category):
    
    dic_patents = related_patents(number_technology , category, carbon_related , display)
    dic_patents_co_inventors = {}
    count = 0
    
    for patent in dic_patents:
        for k in range(len(dic_patents[patent]["list_inventors"])):
            
            dic_patents_co_inventors[count] = {}
            
            
            dic_patents_co_inventors[count]["Latitude"] = dic_patents[patent]["list_inventors"][k]["inventor_latitude"]
            dic_patents_co_inventors[count]["Longitude"] = dic_patents[patent]["list_inventors"][k]["inventor_longitude"]

            count += 1
                
    if dic_patents_co_inventors == {}:
        return "No patent, select other key words"
    
    
    map_df = pd.DataFrame(dic_patents_co_inventors).T
    map_df['Longitude']=map_df['Longitude'].astype(float)
    map_df['Latitude']=map_df['Latitude'].astype(float)



    geometry = [Point(xy) for xy in zip(map_df['Longitude'], map_df['Latitude'])]
    gdf = GeoDataFrame(map_df, geometry=geometry)   

    #this is a simple map that goes with geopandas
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    gdf.plot(ax=world.plot( color='white', edgecolor='black' ), marker='o', color='red', markersize=15 , zorder = 1);
    plt.xlim([-180, 180])
    plt.ylim([-90, 90])


    plt.title("Main inventors: geographic location")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.show()

    

# Results

## Let's look at the papers

### Which papers are related to the technology?

In [38]:
name_file = 'iea'
res = technology("iea", False )
list_categories_tech = []
list_categories = key_words("iea" , False)
list_technologies = [ ( ", ".join(list_categories[i][1]) , i ) for i in range(len(list_categories))  ] 
dic_technologies = {}
for i in range(len(res)):
    names = res[i][1]
    if ", ".join(names) not in list_categories_tech:
        list_categories_tech.append(", ".join(names))
        dic_technologies[", ".join(names)] = []
    dic_technologies[", ".join(names)].append( (", ".join(list_categories[i][1]) , i ))
     

list_climate = [ ("Any related papers" , False ) , ("Climate related papers" , True)]

dic_categories = {}
for elem in list_technologies:
    list_words = elem[0].split(",")[-3:]
    for i in range(len(list_words)):
        if "CCUS" in list_words[i]:
            list_words[i] = list_words[i].replace("CCUS" , "carbon capture storage")
    dic_categories[elem[1]]  = [  ", ".join([ " ".join(words.split()[:3]) for words in list_words ] )  ,  ", ".join([ " ".join(words.split()[:3]) for words in list_words[:-1] ]) , ", ".join([ " ".join(words.split()[:3]) for words in list_words[1:] ] ) ]  
   

                                       
technologies_widget = widgets.Dropdown(options=list_categories_tech, 
                                            description="Choose the category:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));

category_widget = widgets.RadioButtons( values = 1,
                                        description='Papers where the abstract contains:' , 
                                        style = {'description_width':'initial' } , 
                                        layout=Layout(width='1000px'));
                                       
number_technology_widget = widgets.Dropdown(options=list_technologies, 
                                            description="Choose the technology:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));



def update_category(*args):
    category_widget.options = dic_categories[number_technology_widget.value]
    
def update_technology(*args):
    number_technology_widget.options = dic_technologies[technologies_widget.value]
    
                                       
number_technology_widget.observe(update_category)
technologies_widget.observe(update_technology)


print("Which papers are related to the technology?")
interact_manual(get_ranking_related_papers, 
                technologies = technologies_widget,
                number_technology= number_technology_widget ,    
                carbon_related=widgets.Dropdown(options=list_climate, 
                                                description="Select the type of papers:" , 
                                                style = {'description_width':'initial' }   , 
                                                layout=Layout(width='500px')  ) , 
                size = widgets.IntSlider(min=3, 
                                         max=100, 
                                         value=10, 
                                         step=1, 
                                         description="Select the number of papers:" ,  
                                         style = {'description_width':'initial' } , 
                                         layout=Layout(width='500px')),
               research_words = category_widget);


Which papers are related to the technology?


interactive(children=(Dropdown(description='Choose the category:', layout=Layout(width='500px'), options=('Tra…

In [22]:
name_file = 'iea'
res = technology("iea", False )
list_categories_tech = []
dic_technologies = {}
for i in range(len(res)):
    names = res[i][1]
    if ", ".join(names) not in list_categories_tech:
        list_categories_tech.append(", ".join(names))
        dic_technologies[", ".join(names)] = []
    dic_technologies[", ".join(names)].append( (", ".join(list_categories[i][1]) , i ))
     

list_climate = [ ("Any related papers" , False ) , ("Climate related papers" , True)]

                                       
technologies_widget = widgets.Dropdown(options=list_categories_tech, 
                                            description="Choose the category:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));

                                       
number_technology_widget = widgets.Dropdown(options=list_technologies, 
                                            description="Choose the technology:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));


def update_technology(*args):
    number_technology_widget.options = dic_technologies[technologies_widget.value]
    
                                       
technologies_widget.observe(update_technology)


print("Which papers are related to the technology?")
interact_manual(get_ranking_related_papers, 
                technologies = technologies_widget,
                number_technology= number_technology_widget ,    
                carbon_related=widgets.Dropdown(options=list_climate, 
                                                description="Select the type of papers:" , 
                                                style = {'description_width':'initial' }   , 
                                                layout=Layout(width='500px')  ) , 
                size = widgets.IntSlider(min=3, 
                                         max=100, 
                                         value=10, 
                                         step=1, 
                                         description="Select the number of papers:" ,  
                                         style = {'description_width':'initial' } , 
                                         layout=Layout(width='500px')),

               research_words = widgets.Text(value='Enter key words' , description="Papers where the abstract contains your own key words:" , style = {'description_width':'initial' }   , layout=Layout(width='500px'))) ;



Which papers are related to the technology?


interactive(children=(Dropdown(description='Choose the category:', layout=Layout(width='500px'), options=('Tra…

### More data on a technolology?  

In [23]:
name_file = "iea"
res = technology("iea", False )
list_categories = key_words(name_file , False)
list_categories_tech = []
dic_technologies = {}
for i in range(len(res)):
    names = res[i][1]
    if ", ".join(names) not in list_categories_tech:
        list_categories_tech.append(", ".join(names))
        dic_technologies[", ".join(names)] = []
    dic_technologies[", ".join(names)].append( (", ".join(list_categories[i][1]) , i ))
     


                                       
technologies_widget = widgets.Dropdown(options=list_categories_tech, 
                                            description="Choose the category:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));

          
number_technology_widget = widgets.Dropdown(options=list_technologies, 
                                            description="Choose the technology:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));



def update_technology(*args):
    number_technology_widget.options = dic_technologies[technologies_widget.value]
    
                                       
technologies_widget.observe(update_technology)


print("Do you want the quantitative information corresponding to a technology?")
interact_manual(extract_quantitative_data_technology, technologies=technologies_widget , number_technology=number_technology_widget);


Do you want the quantitative information corresponding to a technology?


interactive(children=(Dropdown(description='Choose the category:', layout=Layout(width='500px'), options=('Tra…

### More data on a paper? 

In [24]:

#print("Do you want the quantitative information corresponding to a paper?")
interact_manual(extract_quantitative_data_paper  , work_id= widgets.Text(value='W00000000' , description="Enter a work id from OpenAlex" , style = {'description_width':'initial' }   , layout=Layout(width='500px'))) ;


interactive(children=(Text(value='W00000000', description='Enter a work id from OpenAlex', layout=Layout(width…

### More papers related to the cited projects and universities? 

In [25]:
name_file = "iea"
res = technology("iea", False )
list_categories_tech = []
list_categories = key_words(name_file , False)
dic_technologies = {}
for i in range(len(res)):
    names = res[i][1]
    if ", ".join(names) not in list_categories_tech:
        list_categories_tech.append(", ".join(names))
        dic_technologies[", ".join(names)] = []
    dic_technologies[", ".join(names)].append( (", ".join(list_categories[i][1]) , i ))
     

list_climate = [ ("Any related papers" , False ) , ("Climate related papers" , True)]

                                       
technologies_widget = widgets.Dropdown(options=list_categories_tech, 
                                            description="Choose the category:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));

                                       
number_technology_widget = widgets.Dropdown(options=list_technologies, 
                                            description="Choose the technology:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));


def update_technology(*args):
    number_technology_widget.options = dic_technologies[technologies_widget.value]
    
                                       
technologies_widget.observe(update_technology)

list_climate = [ ("Any related papers" , False ) , ("Climate related papers" , True)]

print("Which papers are related to the projects cited in the technology key initiatives?")
interact_manual(related_projects, technologies=technologies_widget, number_technology=number_technology_widget, carbon_related=widgets.Dropdown(options=list_climate, description="Select the type of papers:" , style = {'description_width':'initial' }   , layout=Layout(width='500px')  ) , size = widgets.IntSlider(min=3, max=100, value=10, step=1, description="Select the number of papers:" ,  style = {'description_width':'initial' } , layout=Layout(width='500px')));


Which papers are related to the projects cited in the technology key initiatives?


interactive(children=(Dropdown(description='Choose the category:', layout=Layout(width='500px'), options=('Tra…

### Who are the main authors? 

In [26]:
res = technology("iea", False )
list_categories_tech = []
list_categories = key_words("iea" , False)
list_technologies = [ ( ", ".join(list_categories[i][1]) , i ) for i in range(len(list_categories))  ] 
dic_technologies = {}
for i in range(len(res)):
    names = res[i][1]
    if ", ".join(names) not in list_categories_tech:
        list_categories_tech.append(", ".join(names))
        dic_technologies[", ".join(names)] = []
    dic_technologies[", ".join(names)].append( (", ".join(list_categories[i][1]) , i ))
     

list_climate = [ ("Any related papers" , False ) , ("Climate related papers" , True)]

dic_categories = {}
for elem in list_technologies:
    list_words = elem[0].split(",")[-3:]
    for i in range(len(list_words)):
        if "CCUS" in list_words[i]:
            list_words[i] = list_words[i].replace("CCUS" , "carbon capture storage")
    dic_categories[elem[1]]  = [  ", ".join([ " ".join(words.split()[:3]) for words in list_words ] )  ,  ", ".join([ " ".join(words.split()[:3]) for words in list_words[:-1] ]) , ", ".join([ " ".join(words.split()[:3]) for words in list_words[1:] ] ) ]  
   

                                       
technologies_widget = widgets.Dropdown(options=list_categories_tech, 
                                            description="Choose the category:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));

category_widget = widgets.RadioButtons( values = 1,
                                        description='Papers where the abstract contains:' , 
                                        style = {'description_width':'initial' } , 
                                        layout=Layout(width='1000px'));
                                       
number_technology_widget = widgets.Dropdown(options=list_technologies, 
                                            description="Choose the technology:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));



def update_category(*args):
    category_widget.options = dic_categories[number_technology_widget.value]
    
def update_technology(*args):
    number_technology_widget.options = dic_technologies[technologies_widget.value]
    
                                       
number_technology_widget.observe(update_category)
technologies_widget.observe(update_technology)

print("Who are the authors who are occuring most frequently in the 100 most related papers?")
interact_manual(main_authors, 
                
                technologies=technologies_widget,
                
                number_technology=number_technology_widget , 
                
                carbon_related=widgets.Dropdown(options=list_climate, 
                                                description="Select the type of papers:" , 
                                                style = {'description_width':'initial' }   , 
                                                layout=Layout(width='500px')  ) , 
                size = widgets.IntSlider(min=3, 
                                         max=100, 
                                         value=10, 
                                         step=1, 
                                         description="Select the number of authors:" ,  
                                         style = {'description_width':'initial' } , 
                                         layout=Layout(width='500px') ),
                
                research_words  =  category_widget);


Who are the authors who are occuring most frequently in the 100 most related papers?


interactive(children=(Dropdown(description='Choose the category:', layout=Layout(width='500px'), options=('Tra…

In [27]:
res = technology("iea", False )
list_categories_tech = []
list_categories = key_words("iea" , False)
dic_technologies = {}
for i in range(len(res)):
    names = res[i][1]
    if ", ".join(names) not in list_categories_tech:
        list_categories_tech.append(", ".join(names))
        dic_technologies[", ".join(names)] = []
    dic_technologies[", ".join(names)].append( (", ".join(list_categories[i][1]) , i ))
     

list_climate = [ ("Any related papers" , False ) , ("Climate related papers" , True)]


                                       
technologies_widget = widgets.Dropdown(options=list_categories_tech, 
                                            description="Choose the category:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));


number_technology_widget = widgets.Dropdown(options=list_technologies, 
                                            description="Choose the technology:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));


def update_technology(*args):
    number_technology_widget.options = dic_technologies[technologies_widget.value]
 
technologies_widget.observe(update_technology)

print("Who are the authors who are occuring most frequently in the 100 most related papers?")
res = interact_manual(main_authors, 
                      
                technologies=technologies_widget,
                      
                number_technology=number_technology_widget , 
                
                carbon_related=widgets.Dropdown(options=list_climate, 
                                                description="Select the type of papers:" , 
                                                style = {'description_width':'initial' }   , 
                                                layout=Layout(width='500px')  ) , 
                size = widgets.IntSlider(min=3, 
                                         max=100, 
                                         value=10, 
                                         step=1, 
                                         description="Select the number of authors:" ,  
                                         style = {'description_width':'initial' } , 
                                         layout=Layout(width='500px') ),
                
                research_words = widgets.Text(value='Enter key words' , description="Papers where the abstract contains your own key words:" , style = {'description_width':'initial' }   , layout=Layout(width='500px'))) 



Who are the authors who are occuring most frequently in the 100 most related papers?


interactive(children=(Dropdown(description='Choose the category:', layout=Layout(width='500px'), options=('Tra…

### Where are the main authors located? 

In [28]:
res = technology("iea", False )
list_categories_tech = []
list_categories = key_words("iea" , False)
list_technologies = [ ( ", ".join(list_categories[i][1]) , i ) for i in range(len(list_categories))  ] 
dic_technologies = {}
for i in range(len(res)):
    names = res[i][1]
    if ", ".join(names) not in list_categories_tech:
        list_categories_tech.append(", ".join(names))
        dic_technologies[", ".join(names)] = []
    dic_technologies[", ".join(names)].append( (", ".join(list_categories[i][1]) , i ))
     

list_climate = [ ("Any related papers" , False ) , ("Climate related papers" , True)]

dic_categories = {}
for elem in list_technologies:
    list_words = elem[0].split(",")[-3:]
    for i in range(len(list_words)):
        if "CCUS" in list_words[i]:
            list_words[i] = list_words[i].replace("CCUS" , "carbon capture storage")
    dic_categories[elem[1]]  = [  ", ".join([ " ".join(words.split()[:3]) for words in list_words ] )  ,  ", ".join([ " ".join(words.split()[:3]) for words in list_words[:-1] ]) , ", ".join([ " ".join(words.split()[:3]) for words in list_words[1:] ] ) ]  
   

                                       
technologies_widget = widgets.Dropdown(options=list_categories_tech, 
                                            description="Choose the category:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));

category_widget = widgets.RadioButtons( values = 1,
                                        description='Papers where the abstract contains:' , 
                                        style = {'description_width':'initial' } , 
                                        layout=Layout(width='1000px'));
                                       
number_technology_widget = widgets.Dropdown(options=list_technologies, 
                                            description="Choose the technology:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));



def update_category(*args):
    category_widget.options = dic_categories[number_technology_widget.value]
    
def update_technology(*args):
    number_technology_widget.options = dic_technologies[technologies_widget.value]
    
                                       
number_technology_widget.observe(update_category)
technologies_widget.observe(update_technology)

print("Where are the authors located?")
interact_manual(map_authors, 
                
                technologies= technologies_widget,
                
                number_technology=number_technology_widget , 
                
                carbon_related=widgets.Dropdown(options=list_climate, 
                                                description="Select the type of papers:" , 
                                                style = {'description_width':'initial' }   , 
                                                layout=Layout(width='500px')  ) , 
                size = widgets.IntSlider(min=3, 
                                         max=30, 
                                         value=10, 
                                         step=1, 
                                         description="Select the number of papers:" ,  
                                         style = {'description_width':'initial' } , 
                                         layout=Layout(width='500px') ),
                
                category  =  category_widget);


Where are the authors located?


interactive(children=(Dropdown(description='Choose the category:', layout=Layout(width='500px'), options=('Tra…

In [29]:
res = technology("iea", False )
list_categories_tech = []
list_categories = key_words("iea" , False)
dic_technologies = {}
for i in range(len(res)):
    names = res[i][1]
    if ", ".join(names) not in list_categories_tech:
        list_categories_tech.append(", ".join(names))
        dic_technologies[", ".join(names)] = []
    dic_technologies[", ".join(names)].append( (", ".join(list_categories[i][1]) , i ))
     

list_climate = [ ("Any related papers" , False ) , ("Climate related papers" , True)]


                                       
technologies_widget = widgets.Dropdown(options=list_categories_tech, 
                                            description="Choose the category:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));


number_technology_widget = widgets.Dropdown(options=list_technologies, 
                                            description="Choose the technology:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));


def update_technology(*args):
    number_technology_widget.options = dic_technologies[technologies_widget.value]
 
technologies_widget.observe(update_technology)

print("Where are the authors located?")
res  =interact_manual(map_authors, 
                      
                technologies= technologies_widget,
                      
                number_technology=number_technology_widget , 
                
                carbon_related=widgets.Dropdown(options=list_climate, 
                                                description="Select the type of papers:" , 
                                                style = {'description_width':'initial' }   , 
                                                layout=Layout(width='500px')  ) , 
                size = widgets.IntSlider(min=3, 
                                         max=30, 
                                         value=10, 
                                         step=1, 
                                         description="Select the number of papers:" ,  
                                         style = {'description_width':'initial' } , 
                                         layout=Layout(width='500px') ),
                
                category  =  widgets.Text(value='Enter key words' , description="Papers where the abstract contains your own key words:" , style = {'description_width':'initial' }   , layout=Layout(width='500px'))) 


Where are the authors located?


interactive(children=(Dropdown(description='Choose the category:', layout=Layout(width='500px'), options=('Tra…

## Let's look at the patents

### Which patents are related to the technology?

In [39]:
res = technology("iea", False )
list_categories_tech = []
list_categories = key_words(name_file , False)
dic_technologies = {}
for i in range(len(res)):
    names = res[i][1]
    if ", ".join(names) not in list_categories_tech:
        list_categories_tech.append(", ".join(names))
        dic_technologies[", ".join(names)] = []
    dic_technologies[", ".join(names)].append( (", ".join(list_categories[i][1]) , i ))
     

list_climate = [ ("Any related patents" , False ) , ("Climate related patents" , True)]

dic_categories = {}
for elem in list_technologies:
    list_words = elem[0].split(",")[-3:]

    dic_categories[elem[1]]  = [  " ".join(list_words[-1].split()[:3]) ,  " ".join(list_words[-2].split()[:3]) if len(list_words) > 1 else "" ,  " ".join(list_words[-3].split()[:3]) if len(list_words) > 2 else ""  ]  
   

                                       
technologies_widget = widgets.Dropdown(options=list_categories_tech, 
                                            description="Choose the category:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));

category_widget = widgets.RadioButtons( values = 1,
                                        description='Patents where the abstract contains:' , 
                                        style = {'description_width':'initial' } , 
                                        layout=Layout(width='1000px'));
                                       
number_technology_widget = widgets.Dropdown(options=list_technologies, 
                                            description="Choose the technology:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));

def update_category(*args):
    category_widget.options = dic_categories[number_technology_widget.value]
    
def update_technology(*args):
    number_technology_widget.options = dic_technologies[technologies_widget.value]
    
                                       
number_technology_widget.observe(update_category)
technologies_widget.observe(update_technology)

print("Which patents are related to the technology?")
interact_manual(get_ranking_patents, 
                technologies=technologies_widget,
                number_technology= number_technology_widget ,    
                carbon_related=widgets.Dropdown(options=list_climate, 
                                                description="Select the type of patents:" , 
                                                style = {'description_width':'initial' }   , 
                                                layout=Layout(width='500px')  ) , 
                size = widgets.IntSlider(min=3, 
                                         max=100, 
                                         value=10, 
                                         step=1, 
                                         description="Select the number of patents:" ,  
                                         style = {'description_width':'initial' } , 
                                         layout=Layout(width='500px')),
               category = category_widget);


Which patents are related to the technology?


interactive(children=(Dropdown(description='Choose the category:', layout=Layout(width='500px'), options=('Tra…

In [31]:
res = technology("iea", False )
list_categories_tech = []
list_categories = key_words(name_file , False)
dic_technologies = {}
for i in range(len(res)):
    names = res[i][1]
    if ", ".join(names) not in list_categories_tech:
        list_categories_tech.append(", ".join(names))
        dic_technologies[", ".join(names)] = []
    dic_technologies[", ".join(names)].append( (", ".join(list_categories[i][1]) , i ))
     

list_climate = [ ("Any related patents" , False ) , ("Climate related patents" , True)]

                                       
technologies_widget = widgets.Dropdown(options=list_categories_tech, 
                                            description="Choose the category:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));

                                       
number_technology_widget = widgets.Dropdown(options=list_technologies, 
                                            description="Choose the technology:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));



def update_category(*args):
    category_widget.options = dic_categories[number_technology_widget.value]
    
def update_technology(*args):
    number_technology_widget.options = dic_technologies[technologies_widget.value]
    
                                       
technologies_widget.observe(update_technology)

print("Which patents are related to the technology?")
interact_manual(get_ranking_patents, 
                technologies=technologies_widget,
                number_technology= number_technology_widget ,    
                carbon_related=widgets.Dropdown(options=list_climate, 
                                                description="Select the type of patents:" , 
                                                style = {'description_width':'initial' }   , 
                                                layout=Layout(width='500px')  ) , 
                size = widgets.IntSlider(min=3, 
                                         max=100, 
                                         value=10, 
                                         step=1, 
                                         description="Select the number of patents:" ,  
                                         style = {'description_width':'initial' } , 
                                         layout=Layout(width='500px')),
                category  =  widgets.Text(value='Enter key words' , description="Patents where the abstract contains your own key words:" , style = {'description_width':'initial' }   , layout=Layout(width='500px'))); 


Which patents are related to the technology?


interactive(children=(Dropdown(description='Choose the category:', layout=Layout(width='500px'), options=('Tra…

### More data on a technology?

In [32]:
res = technology("iea", False )
list_categories = key_words(name_file , False)
list_categories_tech = []
dic_technologies = {}
for i in range(len(res)):
    names = res[i][1]
    if ", ".join(names) not in list_categories_tech:
        list_categories_tech.append(", ".join(names))
        dic_technologies[", ".join(names)] = []
    dic_technologies[", ".join(names)].append( (", ".join(list_categories[i][1]) , i ))
     


                                       
technologies_widget = widgets.Dropdown(options=list_categories_tech, 
                                            description="Choose the category:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));

          
number_technology_widget = widgets.Dropdown(options=list_technologies, 
                                            description="Choose the technology:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));



def update_technology(*args):
    number_technology_widget.options = dic_technologies[technologies_widget.value]
    
                                       
technologies_widget.observe(update_technology)

print("Do you want the quantitative information corresponding to a technology?")
interact_manual(extract_quantitative_data_technology, technologies=technologies_widget, number_technology=number_technology_widget);

Do you want the quantitative information corresponding to a technology?


interactive(children=(Dropdown(description='Choose the category:', layout=Layout(width='500px'), options=('Tra…

### More data on a patent? 

In [33]:
print("Do you want more data on a patent?")
res = technology("iea", False )
list_categories = key_words(name_file , False)
list_categories_tech = []
dic_technologies = {}
for i in range(len(res)):
    names = res[i][1]
    if ", ".join(names) not in list_categories_tech:
        list_categories_tech.append(", ".join(names))
        dic_technologies[", ".join(names)] = []
    dic_technologies[", ".join(names)].append( (", ".join(list_categories[i][1]) , i ))
     


                                       
technologies_widget = widgets.Dropdown(options=list_categories_tech, 
                                            description="Choose the category:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));

          
number_technology_widget = widgets.Dropdown(options=list_technologies, 
                                            description="Choose the technology:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));



def update_technology(*args):
    number_technology_widget.options = dic_technologies[technologies_widget.value]
    
                                       
technologies_widget.observe(update_technology)

#print("Do you want the quantitative information corresponding to a paper?")
interact_manual(extract_quantitative_data_patent, patent_id= widgets.Text(value='1000000' , description="Enter a US patent id from PatentsView" , style = {'description_width':'initial' }   , layout=Layout(width='500px'))) ;


Do you want more data on a patent?


interactive(children=(Text(value='1000000', description='Enter a US patent id from PatentsView', layout=Layout…

### Who are the main inventors? 

In [34]:
res = technology("iea", False )
list_categories_tech = []
list_categories = key_words("iea" , False)
dic_technologies = {}
for i in range(len(res)):
    names = res[i][1]
    if ", ".join(names) not in list_categories_tech:
        list_categories_tech.append(", ".join(names))
        dic_technologies[", ".join(names)] = []
    dic_technologies[", ".join(names)].append( (", ".join(list_categories[i][1]) , i ))
     

list_climate = [ ("Any related patents" , False ) , ("Climate related patents" , True)]

dic_categories = {}
for elem in list_technologies:
    list_words = elem[0].split(",")[-3:]

    dic_categories[elem[1]]  = [  " ".join(list_words[-1].split()[:3]) ,  " ".join(list_words[-2].split()[:3]) if len(list_words) > 1 else "" ,  " ".join(list_words[-3].split()[:3]) if len(list_words) > 2 else ""  ]  
   

                                       
technologies_widget = widgets.Dropdown(options=list_categories_tech, 
                                            description="Choose the category:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));

category_widget = widgets.RadioButtons( values = 1,
                                        description='Patents where the abstract contains:' , 
                                        style = {'description_width':'initial' } , 
                                        layout=Layout(width='1000px'));
                                       
number_technology_widget = widgets.Dropdown(options=list_technologies, 
                                            description="Choose the technology:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));

def update_category(*args):
    category_widget.options = dic_categories[number_technology_widget.value]
    
def update_technology(*args):
    number_technology_widget.options = dic_technologies[technologies_widget.value]
    
                                       
number_technology_widget.observe(update_category)
technologies_widget.observe(update_technology)


print("Who are the inventors who are occuring most frequently in the related patents?")


interact_manual(main_inventors, 
         
        technologies=technologies_widget,
                
         number_technology= number_technology_widget, 
         
         carbon_related=widgets.Dropdown(options=list_climate, 
                                         description="Select the type of papers:" , 
                                         style = {'description_width':'initial' }   , 
                                         layout=Layout(width='500px')  ) ,
        size = widgets.IntSlider(min=3, 
                                         max=100, 
                                         value=10, 
                                         step=1, 
                                         description="Select the number of authors:" ,  
                                         style = {'description_width':'initial' } , 
                                         layout=Layout(width='500px') ),
                
          
         
         category   =  category_widget);



Who are the inventors who are occuring most frequently in the related patents?


interactive(children=(Dropdown(description='Choose the category:', layout=Layout(width='500px'), options=('Tra…

In [35]:
res = technology("iea", False )
list_categories_tech = []
list_categories = key_words("iea" , False)
dic_technologies = {}
for i in range(len(res)):
    names = res[i][1]
    if ", ".join(names) not in list_categories_tech:
        list_categories_tech.append(", ".join(names))
        dic_technologies[", ".join(names)] = []
    dic_technologies[", ".join(names)].append( (", ".join(list_categories[i][1]) , i ))
     

list_climate = [ ("Any related patents" , False ) , ("Climate related patents" , True)]

                                     
technologies_widget = widgets.Dropdown(options=list_categories_tech, 
                                            description="Choose the category:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));


                                       
number_technology_widget = widgets.Dropdown(options=list_technologies, 
                                            description="Choose the technology:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));


def update_technology(*args):
    number_technology_widget.options = dic_technologies[technologies_widget.value]
    
                                       
technologies_widget.observe(update_technology)

print("Who are the inventors who are occuring most frequently in the related patents?")



interact_manual(main_inventors, 
                
        technologies=technologies_widget,
         
         number_technology= number_technology_widget, 
         
         carbon_related=widgets.Dropdown(options=list_climate, 
                                         description="Select the type of papers:" , 
                                         style = {'description_width':'initial' }   , 
                                         layout=Layout(width='500px')  ) ,
        size = widgets.IntSlider(min=3, 
                                         max=100, 
                                         value=10, 
                                         step=1, 
                                         description="Select the number of authors:" ,  
                                         style = {'description_width':'initial' } , 
                                         layout=Layout(width='500px') ),
                
          
         
        category  =  widgets.Text(value='Enter key words' , description="Patents where the abstract contains your own key words:" , style = {'description_width':'initial' }   , layout=Layout(width='500px'))); 






Who are the inventors who are occuring most frequently in the related patents?


interactive(children=(Dropdown(description='Choose the category:', layout=Layout(width='500px'), options=('Tra…

### Where are the main inventors located?

In [36]:
res = technology("iea", False )
list_categories_tech = []
list_categories = key_words("iea" , False)
list_technologies = [ ( ", ".join(list_categories[i][1]) , i ) for i in range(len(list_categories))  ] 
dic_technologies = {}
for i in range(len(res)):
    names = res[i][1]
    if ", ".join(names) not in list_categories_tech:
        list_categories_tech.append(", ".join(names))
        dic_technologies[", ".join(names)] = []
    dic_technologies[", ".join(names)].append( (", ".join(list_categories[i][1]) , i ))
     

list_climate = [ ("Any related patents" , False ) , ("Climate related patents" , True)]

dic_categories = {}
for elem in list_technologies:
    list_words = elem[0].split(",")[-3:]
    dic_categories[elem[1]]  = [  " ".join(list_words[-1].split()[:3]) ,  " ".join(list_words[-2].split()[:3]) if len(list_words) > 1 else "" ,  " ".join(list_words[-3].split()[:3]) if len(list_words) > 2 else ""  ]  
   
                                       
technologies_widget = widgets.Dropdown(options=list_categories_tech, 
                                            description="Choose the category:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));

category_widget = widgets.RadioButtons( values = 1,
                                        description='Patents where the abstract contains:' , 
                                        style = {'description_width':'initial' } , 
                                        layout=Layout(width='1000px'));
                                       
number_technology_widget = widgets.Dropdown(options=list_technologies, 
                                            description="Choose the technology:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));

def update_category(*args):
    category_widget.options = dic_categories[number_technology_widget.value]
    
def update_technology(*args):
    number_technology_widget.options = dic_technologies[technologies_widget.value]
    
                                       
number_technology_widget.observe(update_category)
technologies_widget.observe(update_technology)


print("Where are the inventors located?")

def update_category(*args):
    category_widget.options = dic_categories[number_technology_widget.value]
                                   
number_technology_widget.observe(update_category)


interact_manual(map_inventors, 
                
         technologies=technologies_widget,
         
         number_technology= number_technology_widget, 
         
         carbon_related=widgets.Dropdown(options=list_climate, 
                                         description="Select the type of papers:" , 
                                         style = {'description_width':'initial' }   , 
                                         layout=Layout(width='500px')  ) ,
          
         
         category   =  category_widget);




Where are the inventors located?


interactive(children=(Dropdown(description='Choose the category:', layout=Layout(width='500px'), options=('Tra…

In [37]:
res = technology("iea", False )
list_categories_tech = []
dic_technologies = {}
for i in range(len(res)):
    names = res[i][1]
    if ", ".join(names) not in list_categories_tech:
        list_categories_tech.append(", ".join(names))
        dic_technologies[", ".join(names)] = []
    dic_technologies[", ".join(names)].append( (", ".join(list_categories[i][1]) , i ))
     

list_climate = [ ("Any related patents" , False ) , ("Climate related patents" , True)]

                             
technologies_widget = widgets.Dropdown(options=list_categories_tech, 
                                            description="Choose the category:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));

number_technology_widget = widgets.Dropdown(options=list_technologies, 
                                            description="Choose the technology:" , 
                                            style = {'description_width':'initial' } , 
                                            layout=Layout(width='500px'));


def update_technology(*args):
    number_technology_widget.options = dic_technologies[technologies_widget.value]
    
                                       
technologies_widget.observe(update_technology)


print("Where are the inventors located?")

def update_category(*args):
    category_widget.options = dic_categories[number_technology_widget.value]
                                   
number_technology_widget.observe(update_category)


interact_manual(map_inventors, 
                
        technologies=technologies_widget,
         
         number_technology= number_technology_widget, 
         
         carbon_related=widgets.Dropdown(options=list_climate, 
                                         description="Select the type of papers:" , 
                                         style = {'description_width':'initial' }   , 
                                         layout=Layout(width='500px')  ) ,
          
         
        category  =  widgets.Text(value='Enter key words' , description="Patents where the abstract contains your own key words:" , style = {'description_width':'initial' }   , layout=Layout(width='500px'))); 



Where are the inventors located?


interactive(children=(Dropdown(description='Choose the category:', layout=Layout(width='500px'), options=('Tra…